In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential

In [3]:
import json
from sklearn.model_selection import train_test_split

with open('data_json/SubtaskA/subtaskA_train_monolingual.jsonl', 'r') as f:
    df = pd.read_json(f, lines=True, orient='records')

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    df['text'], df['label'], test_size=0.2, random_state=42)


In [4]:
# Tokenize the text
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
padded_sequences = pad_sequences(sequences, maxlen=100, truncating='post')


In [5]:
# Define the model
model = Sequential()
model.add(Embedding(10000, 32, input_length=100))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

In [6]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(padded_sequences, y_train, epochs=5, verbose=1)

Epoch 1/5
2994/2994 [==============================] - 430s 132ms/step - loss: 0.4464 - accuracy: 0.7938
Epoch 2/5
2994/2994 [==============================] - 394s 131ms/step - loss: 0.3642 - accuracy: 0.8399
Epoch 3/5
2994/2994 [==============================] - 402s 134ms/step - loss: 0.3303 - accuracy: 0.8553
Epoch 4/5
2994/2994 [==============================] - 398s 133ms/step - loss: 0.2996 - accuracy: 0.8718
Epoch 5/5
2994/2994 [==============================] - 397s 133ms/step - loss: 0.2725 - accuracy: 0.8835


In [7]:
# Get the weights of the embedding layer
embedding_weights = model.get_weights()[0]

# Get the word index from the tokenizer
word_index = tokenizer.word_index

# Create a list of (word, weight) pairs
word_weights = []
for word, index in word_index.items():
    if index < embedding_weights.shape[0]:
        weight = np.linalg.norm(embedding_weights[index])
        word_weights.append((word, weight))

# Store the weights in a csv file
sorted_features = sorted(word_weights, key=lambda x: x[1])
weights_df = pd.DataFrame(sorted_features, columns=['word', 'weight'])
weights_df.to_csv('RNN_outputs/RNN_weights.csv', index=False)


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(dev_df['label'], dev_labels))
